<a href="https://colab.research.google.com/github/ZhangTaTa/keras_learning/blob/master/img_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling2D, Flatten, Conv2D
from keras.optimizers import Adam
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [0]:
np.random.seed(1337)  # for reproducibility

更新TensorFlow到最新版

In [0]:
import tensorflow as tf
tf.__version__

'2.2.0-rc4'

In [0]:
# !pip install --upgrade tensorflow

加载数据

In [0]:
X = np.load("imgDataMatrix.npy", allow_pickle=True)
Y = np.load("imgNameMatrix.npy", allow_pickle=True)

In [0]:
print(Y.shape)
print(X.shape)

(3647, 1)
(3647, 30000)


随机分类train和test（8:2）

In [0]:
ran = np.random.random(size=(len(Y), 1))
trainList = ran > 0.2
testList = ran <= 0.2
dataRow = np.concatenate((X, Y, trainList), axis=1)
X_train = dataRow[dataRow[:, X.shape[1]+1] == True][:, 0:X.shape[1]]
Y_train = dataRow[dataRow[:, X.shape[1]+1] == True][:, X.shape[1]]
X_test = dataRow[dataRow[:, X.shape[1]+1] == False][:, 0:X.shape[1]]
Y_test = dataRow[dataRow[:, X.shape[1]+1] == False][:, X.shape[1]]

CNN模型

In [0]:
# 数据预处理（size：100×100，RGB格式）
X_train = X_train.reshape(-1, 3, 100, 100) / 255.
X_test = X_test.reshape(-1, 3, 100, 100) / 255.
Y_train = np_utils.to_categorical(Y_train, num_classes=2)
Y_test = np_utils.to_categorical(Y_test, num_classes=2)
# 输入改成(100, 100, 3) 手动channels_last
X_train = np.moveaxis(X_train, 1, -1)
X_test = np.moveaxis(X_test, 1, -1)

In [0]:
print(Y_test.shape)
print(X_test.shape)

(700, 2)
(700, 100, 100, 3)


In [0]:
# 建立模型
model = Sequential()

# Conv layer 1; output shape (32, 96, 96)
model.add(Conv2D(
    input_shape=(100, 100, 3),
    filters=32,   # 过滤器个数
    kernel_size=5,    # 卷积核尺寸
    padding='same',   # Padding method 避免数据丢失的东西
    activation='relu'))

# Pooling layer 1 (max pooling); output shape (32, 48, 48)
model.add(MaxPooling2D(
    pool_size=(2, 2)))

# Conv layer 2; output shape (64, 44, 44)
model.add(Conv2D(
    filters=64,
    kernel_size=5,
    padding='same',
    activation='relu'))

# Pooling layer 2 (max pooling); output shape (64, 22, 22)
model.add(MaxPooling2D(
    pool_size=(2, 2)))

# Fully connected layer 1 input shape (64 * 22 * 22) = (30976), output shape (1024)
model.add(Flatten())    # 将输入展平
model.add(Dense(units=1024, activation='relu'))

# Fully connected layer 1 input shape (1024), output shape (2)
model.add(Dense(units=2, activation='relu'))

# Another way to define your optimizer
adam = Adam(lr=3e-5)  # 一种自适应的梯度下降优化器 https://www.jianshu.com/p/aebcaf8af76e

# We add metrics to get more results you want to see
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# 训练模型
print('Training ------------')
# Another way to train the model
model.fit(X_train, Y_train, epochs=5, batch_size=64)

print('\nTesting ------------')
# Evaluate the model with the metrics we defined earlier
loss, accuracy = model.evaluate(X_test, Y_test)

print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

Training ------------
Epoch 1/5
2947/2947 [==============================] - 3s 1ms/step - loss: 0.8477 - accuracy: 0.6834
Epoch 2/5
2947/2947 [==============================] - 2s 548us/step - loss: 0.5338 - accuracy: 0.7533
Epoch 3/5
2947/2947 [==============================] - 2s 554us/step - loss: 0.4857 - accuracy: 0.7832
Epoch 4/5
2947/2947 [==============================] - 2s 556us/step - loss: 0.4461 - accuracy: 0.8045
Epoch 5/5
2947/2947 [==============================] - 2s 549us/step - loss: 0.4405 - accuracy: 0.8096

Testing ------------
700/700 [==============================] - 0s 387us/step

test loss:  0.567700183050973

test accuracy:  0.8042857050895691
